<a href="https://colab.research.google.com/github/jiweigang1/claude-cookbooks/blob/main/tool_use/programmatic_tool_calling_ptc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使用Claude API进行程序化工具调用(PTC)

程序化工具调用(PTC)允许Claude在代码执行环境中编写代码来程序化地调用工具，而不是要求模型为每个工具调用进行往返调用。这大大减少了多个工具调用的端到端延迟，并且可以通过允许模型编写代码来显著减少token消耗，这些代码可以在到达模型上下文窗口之前移除不相关的上下文（例如，通过在大型且嘈杂的文件中搜索关键信息）。

当面对第三方API和您可能无法直接修改的工具时，PTC可以通过允许Claude编写代码来帮助减少上下文使用，这些代码可以在代码执行环境中被调用。

在本教程中，我们将使用一个团队费用管理的模拟API。该API设计为需要多次调用，并将返回大量结果，这有助于说明程序化工具调用的好处。

## 在本教程结束时，您将能够：

- 理解常规工具调用和程序化工具调用(PTC)之间的区别
- 编写利用PTC的智能代理

## 前提条件

在开始学习本指南之前，请确保您具备：

**所需知识**

- Python基础知识 - 熟悉async/await、函数和基本数据结构
- 对智能代理模式和工具调用的基本理解

**所需工具**

- Python 3.11或更高版本
- Anthropic API密钥
- Anthropic Python SDK >= 0.72

## 设置

首先，安装所需的依赖项：

In [4]:
# %pip install -r requirements.txt

注意：确保您的.env文件包含：

`ANTHROPIC_API_KEY=your_key_here`

加载环境变量并配置客户端。我们还加载了一个辅助工具来可视化Claude消息响应。

In [5]:
from dotenv import load_dotenv
from utils.visualize import visualize

load_dotenv()

MODEL = "claude-sonnet-4-5"

viz = visualize(auto_show=True)

ModuleNotFoundError: No module named 'utils'

## 理解第三方API

在[utils/team_expense_api.py](utils/team_expense_api.py)中，定义了三个函数：`get_team_members`、`get_expenses`和`get_custom_budget`。`get_team_members`函数允许我们检索给定部门的所有员工及其角色、级别和联系信息。`get_expenses`函数返回员工在特定季度的所有费用明细项目——每个员工可能有数百条记录，每条记录包含大量元数据，包括收据URL、审批链、商户详细信息等。`get_custom_budget`函数检查特定员工是否有自定义差旅预算例外（否则他们使用标准的5000美元季度限额）。

在这种情况下，我们需要分析团队费用并识别哪些员工超出了预算。传统上，我们可能会手动为每个人拉取费用报告，按类别汇总费用，对照预算限额进行比较（检查自定义预算例外），并编制报告。相反，我们将要求Claude为我们执行此分析，使用可用工具检索团队数据，获取可能包含丰富元数据的数百条费用明细项目，并确定谁超出了预算。

这里的关键挑战是，每个员工可能有100多条费用明细项目需要获取、解析和汇总——而且`get_custom_budget`工具只能在分析费用以查看是否有人超出标准预算后才能调用。这创建了一个顺序依赖链，使其成为演示程序化工具调用好处的理想用例。

我们将把工具定义传递给消息API，并要求Claude执行分析。如果您不熟悉Claude API的工具使用方式，请阅读[实现工具使用](https://docs.claude.com/en/docs/agents-and-tools/tool-use/implement-tool-use)的文档。

In [ ]:
import json

import anthropic
from utils.team_expense_api import get_custom_budget, get_expenses, get_team_members

client = anthropic.Anthropic()

# Tool definitions for the team expense API
tools = [
    {
        "name": "get_team_members",
        "description": 'Returns a list of team members for a given department. Each team member includes their ID, name, role, level (junior, mid, senior, staff, principal), and contact information. Use this to get a list of people whose expenses you want to analyze. Available departments are: engineering, sales, and marketing.\n\nRETURN FORMAT: Returns a JSON string containing an ARRAY of team member objects (not wrapped in an outer object). Parse with json.loads() to get a list. Example: [{"id": "ENG001", "name": "Alice", ...}, {"id": "ENG002", ...}]',
        "input_schema": {
            "type": "object",
            "properties": {
                "department": {
                    "type": "string",
                    "description": "The department name. Case-insensitive.",
                }
            },
            "required": ["department"],
        },
        "input_examples": [
            {"department": "engineering"},
            {"department": "sales"},
            {"department": "marketing"},
        ],
    },
    {
        "name": "get_expenses",
        "description": "Returns all expense line items for a given employee in a specific quarter. Each expense includes extensive metadata: date, category, description, amount (in USD), currency, status (approved, pending, rejected), receipt URL, approval chain, merchant name and location, payment method, and project codes. An employee may have 20-50+ expense line items per quarter, and each line item contains substantial metadata for audit and compliance purposes. Categories include: 'travel' (flights, trains, rental cars, taxis, parking), 'lodging' (hotels, airbnb), 'meals', 'software', 'equipment', 'conference', 'office', and 'internet'. IMPORTANT: Only expenses with status='approved' should be counted toward budget limits.\n\nRETURN FORMAT: Returns a JSON string containing an ARRAY of expense objects (not wrapped in an outer object with an 'expenses' key). Parse with json.loads() to get a list directly. Example: [{\"expense_id\": \"ENG001_Q3_001\", \"amount\": 1250.50, \"category\": \"travel\", ...}, {...}]",
        "input_schema": {
            "type": "object",
            "properties": {
                "employee_id": {
                    "type": "string",
                    "description": "The unique employee identifier",
                },
                "quarter": {
                    "type": "string",
                    "description": "Quarter identifier: 'Q1', 'Q2', 'Q3', or 'Q4'",
                },
            },
            "required": ["employee_id", "quarter"],
        },
        "input_examples": [
            {"employee_id": "ENG001", "quarter": "Q3"},
            {"employee_id": "SAL002", "quarter": "Q1"},
            {"employee_id": "MKT001", "quarter": "Q4"},
        ],
    },
    {
        "name": "get_custom_budget",
        "description": 'Get the custom quarterly travel budget for a specific employee. Most employees have a standard $5,000 quarterly travel budget. However, some employees have custom budget exceptions based on their role requirements. This function checks if a specific employee has a custom budget assigned.\n\nRETURN FORMAT: Returns a JSON string containing a SINGLE OBJECT (not an array). Parse with json.loads() to get a dict. Example: {"user_id": "ENG001", "has_custom_budget": false, "travel_budget": 5000, "reason": "Standard", "currency": "USD"}',
        "input_schema": {
            "type": "object",
            "properties": {
                "user_id": {
                    "type": "string",
                    "description": "The unique employee identifier",
                }
            },
            "required": ["user_id"],
        },
        "input_examples": [
            {"user_id": "ENG001"},
            {"user_id": "SAL002"},
            {"user_id": "MKT001"},
        ],
    },
]

tool_functions = {
    "get_team_members": get_team_members,
    "get_expenses": get_expenses,
    "get_custom_budget": get_custom_budget,
}

## 传统工具调用（基线）

在第一个示例中，我们将使用传统工具调用来建立基线。

我们将使用初始查询调用`messages.create` API。当模型因`tool_use`原因停止时，我们将按请求执行工具，然后将工具输出添加到消息中，再次调用模型。

In [ ]:
import time

from anthropic.types import TextBlock, ToolUseBlock
from anthropic.types.beta import (
    BetaMessageParam as MessageParam,
)
from anthropic.types.beta import (
    BetaTextBlock,
    BetaToolUseBlock,
)

messages: list[MessageParam] = []


def run_agent_without_ptc(user_message):
    """Run agent using traditional tool calling"""
    messages.append({"role": "user", "content": user_message})
    total_tokens = 0
    start_time = time.time()
    api_counter = 0

    while True:
        response = client.beta.messages.create(
            model=MODEL,
            max_tokens=4000,
            tools=tools,
            messages=messages,
            betas=["advanced-tool-use-2025-11-20"],
        )

        api_counter += 1

        # Track token usage
        total_tokens += response.usage.input_tokens + response.usage.output_tokens
        viz.capture(response)
        if response.stop_reason == "end_turn":
            # Extract the first text block from the response
            final_response = next(
                (
                    block.text
                    for block in response.content
                    if isinstance(block, (BetaTextBlock, TextBlock))
                ),
                None,
            )
            elapsed_time = time.time() - start_time
            return final_response, messages, total_tokens, elapsed_time, api_counter

        # Process tool calls
        if response.stop_reason == "tool_use":
            # First, add the assistant's response to messages
            messages.append({"role": "assistant", "content": response.content})

            # Collect all tool results
            tool_results = []

            for block in response.content:
                if isinstance(block, (BetaToolUseBlock, ToolUseBlock)):
                    tool_name = block.name
                    tool_input = block.input
                    tool_use_id = block.id

                    result = tool_functions[tool_name](**tool_input)

                    content = str(result)

                    tool_result = {
                        "type": "tool_result",
                        "tool_use_id": tool_use_id,
                        "content": content,
                    }
                    tool_results.append(tool_result)

            # Append all tool results at once after collecting them
            messages.append({"role": "user", "content": tool_results})

        else:
            print(f"\nUnexpected stop reason: {response.stop_reason}")
            elapsed_time = time.time() - start_time

            final_response = next(
                (
                    block.text
                    for block in response.content
                    if isinstance(block, (BetaTextBlock, TextBlock))
                ),
                f"Stopped with reason: {response.stop_reason}",
            )
            return final_response, messages, total_tokens, elapsed_time, api_counter

我们对模型的初始查询提供了一些说明来帮助指导模型。为简洁起见，我们要求模型每个工具只调用一次。对于更深入的调查，模型可能希望查看多个系统或时间段。

In [ ]:
query = "Which engineering team members exceeded their Q3 travel budget? Standard quarterly travel budget is $5,000. However, some employees have custom budget limits. For anyone who exceeded the $5,000 standard budget, check if they have a custom budget exception. If they do, use that custom limit instead to determine if they truly exceeded their budget."

In [ ]:
# Run the agent
result, conversation, total_tokens, elapsed_time, api_count_without_ptc = run_agent_without_ptc(
    query
)

print(f"Result: {result}")
print(f"API calls made: {api_count_without_ptc}")
print(f"Total tokens used: {total_tokens:,}")
print(f"Total time taken: {elapsed_time:.2f}s")

╭────────────────────────────────────────────── Claude API Response ──────────────────────────────────────────────╮
│ Claude Message (assistant) │ tokens: 1,859 in • 85 out • 1,944 total                                            │
│ ├── Model: claude-sonnet-4-5-20250929                                                                           │
│ ├── Stop Reason: tool_use                                                                                       │
│ └── Content (2 blocks)                                                                                          │
│     ├── Block 1                                                                                                 │
│     │   └── Text                                                                                                │
│     │       └── I'll help you identify which engineering team members exceeded their Q3 travel budget. Let me   │
│     │           start by getting the list of engineering team members.                                          │
│     └── Block 2                                                                                                 │
│         └── Tool Use: get_team_members                                                                          │
│             ├── ID: toolu_01LuouuJYp1sSvBe2Du7EG7v                                                              │
│             ├── Caller: model (direct)                                                                          │
│             └── Input:                                                                                          │
│                 └── {                                                                                           │
│                       "department": "engineering"                                                               │
│                     }                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── Claude API Response ───────────────────────────────╮
│ Claude Message (assistant) │ tokens: 2,473 in • 497 out • 2,970 total            │
│ ├── Model: claude-sonnet-4-5-20250929                                            │
│ ├── Stop Reason: tool_use                                                        │
│ └── Content (9 blocks)                                                           │
│     ├── Block 1                                                                  │
│     │   └── Text                                                                 │
│     │       └── Now let me get the Q3 expenses for all engineering team members: │
│     ├── Block 2                                                                  │
│     │   └── Tool Use: get_expenses                                               │
│     │       ├── ID: toolu_01Wu8LLTT2sKTTqpVwGT65Lj                               │
│     │       ├── Caller: model (direct)                                           │
│     │       └── Input:                                                           │
│     │           └── {                                                            │
│     │                 "employee_id": "ENG001",                                   │
│     │                 "quarter": "Q3"                                            │
│     │               }                                                            │
│     ├── Block 3                                                                  │
│     │   └── Tool Use: get_expenses                                               │
│     │       ├── ID: toolu_01KzjQ5mQJa9ocWjCGzYkD9F                               │
│     │       ├── Caller: model (direct)                                           │
│     │       └── Input:                                                           │
│     │           └── {                                                            │
│     │                 "employee_id": "ENG002",                                   │
│     │                 "quarter": "Q3"                                            │
│     │               }                                                            │
│     ├── Block 4                                                                  │
│     │   └── Tool Use: get_expenses                                               │
│     │       ├── ID: toolu_01RjjhZTg9JsKXE5E9S6Foho                               │
│     │       ├── Caller: model (direct)                                           │
│     │       └── Input:                                                           │
│     │           └── {                                                            │
│     │                 "employee_id": "ENG003",                                   │
│     │                 "quarter": "Q3"                                            │
│     │               }                                                            │
│     ├── Block 5                                                                  │
│     │   └── Tool Use: get_expenses                                               │
│     │       ├── ID: toolu_013xqpxpfc2N9rP5W5uMLAo9                               │
│     │       ├── Caller: model (direct)                                           │
│     │       └── Input:                                                           │
│     │           └── {                                                            │
│     │                 "employee_id": "ENG004",                                   │
│     │                 "quarter": "Q3"                                            │
│     │               }                                                            │
│     ├── Block 6                                                                  │
│     │   └── Tool Use: get_expenses                                               │
│     │       ├── ID: toolu_019zfzG6Wox8iDqy1dUXiH3t                               │
│    

╭────────────────────────────────────────────── Claude API Response ──────────────────────────────────────────────╮
│ Claude Message (assistant) │ tokens: 51,744 in • 290 out • 52,034 total                                         │
│ ├── Model: claude-sonnet-4-5-20250929                                                                           │
│ ├── Stop Reason: tool_use                                                                                       │
│ └── Content (7 blocks)                                                                                          │
│     ├── Block 1                                                                                                 │
│     │   └── Text                                                                                                │
│     │       └── Now let me calculate the approved travel expenses for each engineer and identify who exceeded   │
│     │           $5,000:                                                                                         │
│     ├── Block 2                                                                                                 │
│     │   └── Tool Use: get_custom_budget                                                                         │
│     │       ├── ID: toolu_013oegKwjvToLwEW1daDD8av                                                              │
│     │       ├── Caller: model (direct)                                                                          │
│     │       └── Input:                                                                                          │
│     │           └── {                                                                                           │
│     │                 "user_id": "ENG001"                                                                       │
│     │               }                                                                                           │
│     ├── Block 3                                                                                                 │
│     │   └── Tool Use: get_custom_budget                                                                         │
│     │       ├── ID: toolu_0162W4Ycr9FcVVED65exjAj4                                                              │
│     │       ├── Caller: model (direct)                                                                          │
│     │       └── Input:                                                                                          │
│     │           └── {                                                                                           │
│     │                 "user_id": "ENG003"                                                                       │
│     │               }                                                                                           │
│     ├── Block 4                                                                                                 │
│     │   └── Tool Use: get_custom_budget                                                                         │
│     │       ├── ID: toolu_01JcTX5rnwFxA99Am33gXmh6                                                              │
│     │       ├── Caller: model (direct)                                                                          │
│     │       └── Input:                                                                                          │
│     │           └── {                                                                                           │
│     │                 "user_id": "ENG005"                                                                       │
│     │               }                                                                                           │
│     ├── Block 5                                                                                                 │
│     │   └── Tool Use: get_custom_budget               

╭────────────────────────────────────────────── Claude API Response ──────────────────────────────────────────────╮
│ Claude Message (assistant) │ tokens: 52,533 in • 992 out • 53,525 total                                         │
│ ├── Model: claude-sonnet-4-5-20250929                                                                           │
│ ├── Stop Reason: end_turn                                                                                       │
│ └── Content (1 blocks)                                                                                          │
│     └── Block 1                                                                                                 │
│         └── Text                                                                                                │
│             └── Now let me analyze the data. I'll calculate the approved travel expenses for each engineer:     │
│                                                                                                                 │
│                 **Analysis of Q3 Travel Expenses:**                                                             │
│                                                                                                                 │
│                 **ENG001 - Alice Chen (Senior Software Engineer)**                                              │
│                 - Approved travel expenses: $1,161.04 + $18.63 + $13.21 + $36.55 + $1,440.42 + $166.46 + $48.43 │
│                 + $1,124.56 + $1,245.90 + $1,498.42 = **$6,753.62**                                             │
│                 - Budget: $5,000 (Standard)                                                                     │
│                 - **EXCEEDED by $1,753.62** ❌                                                                  │
│                                                                                                                 │
│                 **ENG002 - Bob Martinez (Staff Engineer)**                                                      │
│                 - Approved travel expenses: $180.16 + $10.07 + $20.76 = **$210.99**                             │
│                 - Budget: $5,000 (Standard)                                                                     │
│                 - Under budget ✓                                                                                │
│                                                                                                                 │
│                 **ENG003 - Carol White (Software Engineer)**                                                    │
│                 - Approved travel expenses: $24.75 + $424.74 + $1,397.17 + $1,026.12 + $1,288.36 + $1,128.90 +  │
│                 $1,148.42 + $45.03 = **$6,483.49**                                                              │
│                 - Budget: $5,000 (Standard)                                                                     │
│                 - **EXCEEDED by $1,483.49** ❌                                                                  │
│                                                                                                                 │
│                 **ENG004 - David Kim (Principal Engineer)**                                                     │
│                 - Approved travel expenses: $21.68 + $46.12 + $1,008.68 + $46.43 = **$1,122.91**                │
│                 - Budget: $5,000 (Standard)                                                                     │
│                 - Under budget ✓                                                                                │
│                                                                                                                 │
│                 **ENG005 - Emma Johnson (Junior Software Engineer)                                              │
│                 ... (truncated)                         

Result: Now let me analyze the data. I'll calculate the approved travel expenses for each engineer:

**Analysis of Q3 Travel Expenses:**

**ENG001 - Alice Chen (Senior Software Engineer)**
- Approved travel expenses: $1,161.04 + $18.63 + $13.21 + $36.55 + $1,440.42 + $166.46 + $48.43 + $1,124.56 + $1,245.90 + $1,498.42 = **$6,753.62**
- Budget: $5,000 (Standard)
- **EXCEEDED by $1,753.62** ❌

**ENG002 - Bob Martinez (Staff Engineer)**
- Approved travel expenses: $180.16 + $10.07 + $20.76 = **$210.99**
- Budget: $5,000 (Standard)
- Under budget ✓

**ENG003 - Carol White (Software Engineer)**
- Approved travel expenses: $24.75 + $424.74 + $1,397.17 + $1,026.12 + $1,288.36 + $1,128.90 + $1,148.42 + $45.03 = **$6,483.49**
- Budget: $5,000 (Standard)
- **EXCEEDED by $1,483.49** ❌

**ENG004 - David Kim (Principal Engineer)**
- Approved travel expenses: $21.68 + $46.12 + $1,008.68 + $46.43 = **$1,122.91**
- Budget: $5,000 (Standard)
- Under budget ✓

**ENG005 - Emma Johnson (Junior Software E

很好！我们可以看到Claude能够成功使用可用工具来识别哪些团队成员超出了差旅预算。然而，我们也可以看到我们为了完成这项任务使用了大量token。Claude必须通过其上下文窗口摄取所有费用明细项目——每个员工可能有100多条记录，每条记录包含大量元数据，包括收据URL、审批链、商户信息等——以便解析它们，按类别汇总总计，并对照预算限额进行比较。

此外，传统工具调用方法需要多个顺序往返：首先获取团队成员，然后获取每个人的费用，然后检查超出标准限额的人的自定义预算。每次往返都会增加延迟，所有来自费用记录的丰富元数据都会流经模型的上下文。

让我们看看是否可以使用PTC通过允许Claude编写代码在代码执行环境中处理这些大型数据集来改善性能。

要在工具上启用PTC，我们必须首先向应该可通过代码执行调用的任何工具添加`allowed_callers`字段。

**需要考虑的关键点**

- 没有allowed_callers的工具默认为仅模型调用
- 通过在调用者中包含多个调用者，工具可以由模型和代码执行调用：`["direct", "code_execution_20250825"]`
- 仅选择那些可以安全地进行程序化/重复执行的工具。

In [ ]:
import copy

ptc_tools = copy.deepcopy(tools)
for tool in ptc_tools:
    tool["allowed_callers"] = ["code_execution_20250825"]  # type: ignore


# Add the code execution tool
ptc_tools.append(
    {
        "type": "code_execution_20250825",  # type: ignore
        "name": "code_execution",
    }
)

现在我们已更新工具定义以允许程序化工具调用，我们可以使用PTC运行我们的智能代理。为此，我们必须对函数进行一些更改。我们必须使用`beta`消息API。

1. 我们已将`"advanced-tool-use-2025-11-20"`添加到betas中。
2. 如果定义了`container_id`，我们会将其与请求一起传递。这仅对有状态工作流程（如我们的工作流程）是必要的。在单轮工作流程中，这不是必需的。
3. 我们可以检查`tool_use`块中的`caller`字段，以确定此工具调用是来自直接模型调用还是来自程序化调用。

请注意，在任何情况下，我们都通过Claude API发送我们的工具结果，然而只有`direct`调用会被模型"看到"。`code_execution_20250825`类型只会被代码执行容器看到。

In [ ]:
messages = []


def run_agent_with_ptc(user_message):
    """Run agent using PTC"""
    messages.append({"role": "user", "content": user_message})
    total_tokens = 0
    start_time = time.time()
    container_id = None
    api_counter = 0

    while True:
        # Build request with PTC beta headers
        request_params = {
            "model": MODEL,
            "max_tokens": 4000,
            "tools": ptc_tools,
            "messages": messages,
        }

        response = client.beta.messages.create(
            **request_params,
            betas=[
                "advanced-tool-use-2025-11-20",
            ],
            extra_body={"container": container_id} if container_id else None,
        )
        viz.capture(response)
        api_counter += 1

        # Track container for stateful execution
        if hasattr(response, "container") and response.container:
            container_id = response.container.id
            print(f"\n[Container] ID: {container_id}")
            if hasattr(response.container, "expires_at"):
                # If the container has expired, we would need to restart our workflow. In our case, it completes before expiration.
                print(f"[Container] Expires at: {response.container.expires_at}")

        # Track token usage
        total_tokens += response.usage.input_tokens + response.usage.output_tokens

        if response.stop_reason == "end_turn":
            # Extract the first text block from the response
            final_response = next(
                (block.text for block in response.content if isinstance(block, BetaTextBlock)),
                None,
            )
            elapsed_time = time.time() - start_time
            return final_response, messages, total_tokens, elapsed_time, api_counter

        # As before, we process tool calls
        if response.stop_reason == "tool_use":
            # First, add the assistant's response to messages
            messages.append({"role": "assistant", "content": response.content})

            # Collect all tool results
            tool_results = []

            for block in response.content:
                if isinstance(block, BetaToolUseBlock):
                    tool_name = block.name
                    tool_input = block.input
                    tool_use_id = block.id

                    # We can use caller type to understand how the tool was invoked
                    caller_type = block.caller["type"]  # type: ignore

                    if caller_type == "code_execution_20250825":
                        print(f"[PTC] Tool called from code execution environment: {tool_name}")

                    elif caller_type == "direct":
                        print(f"[Direct] Tool called by model: {tool_name}")

                    result = tool_functions[tool_name](**tool_input)

                    # Format result as proper content for the API
                    if isinstance(result, list) and result and isinstance(result[0], str):
                        content = "\n".join(result)
                    elif isinstance(result, (dict, list)):
                        content = json.dumps(result)
                    else:
                        content = str(result)

                    tool_results.append(
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use_id,
                            "content": content,
                        }
                    )

            messages.append({"role": "user", "content": tool_results})

        else:
            print(f"\nUnexpected stop reason: {response.stop_reason}")
            elapsed_time = time.time() - start_time

            final_response = next(
                (block.text for block in response.content if isinstance(block, BetaTextBlock)),
                f"Stopped with reason: {response.stop_reason}",
            )
            return final_response, messages, total_tokens, elapsed_time, api_counter

In [ ]:
# Run the PTC agent
result_ptc, conversation_ptc, total_tokens_ptc, elapsed_time_ptc, api_count_with_ptc = (
    run_agent_with_ptc(query)
)

╭────────────────────────────────────────────── Claude API Response ──────────────────────────────────────────────╮
│ Claude Message (assistant) │ tokens: 4,134 in • 539 out • 4,673 total                                           │
│ ├── Model: claude-sonnet-4-5-20250929                                                                           │
│ ├── Stop Reason: tool_use                                                                                       │
│ └── Content (3 blocks)                                                                                          │
│     ├── Block 1                                                                                                 │
│     │   └── Text                                                                                                │
│     │       └── I'll help you identify which engineering team members exceeded their Q3 travel budget. Let me   │
│     │           start by getting the engineering team members and their expenses.                               │
│     ├── Block 2                                                                                                 │
│     │   └── Server Tool Use                                                                                     │
│     │       ├── ID: srvtoolu_015mWPqaFni4B313UieCxbny                                                           │
│     │       ├── Caller: direct                                                                                  │
│     │       └── Code:                                                                                           │
│     │           └──    1                                                                                        │
│     │                  2 import asyncio                                                                         │
│     │                  3 import json                                                                            │
│     │                  4                                                                                        │
│     │                  5 async def main():                                                                      │
│     │                  6     # First, get all engineering team members                                          │
│     │                  7     team_members_json = await get_team_members({'department': 'engineering'})          │
│     │                  8     team_members = json.loads(team_members_json)                                       │
│     │                  9                                                                                        │
│     │                 10     print(f"Found {len(team_members)} engineering team members")                       │
│     │                 11                                                                                        │
│     │                 12     # Get Q3 expenses for all team members in parallel                                 │
│     │                 13     expense_tasks = [                                                                  │
│     │                 14         get_expenses({'employee_id': member['id'], 'quarter': 'Q3'})                   │
│     │                 15         for member in team_members                                                     │
│     │                 16     ]                                                                                  │
│     │                 17                                                                                        │
│     │                 18     expenses_results = await asyncio.gather(*expense_tasks)                            │
│     │                 19                                                                                        │
│     │                 20     # Calculate travel expenses for each member                                        │
│     │                 21     travel_spending = {}     


[Container] ID: container_011CVSAwq5J4vNPi3A4P2Rwh
[Container] Expires at: 2025-11-24 05:41:17.467494+00:00
[PTC] Tool called from code execution environment: get_team_members


╭──────────────────── Claude API Response ────────────────────╮
│ Claude Message (assistant) │ tokens: 0 in • 0 out • 0 total │
│ ├── Model: claude-sonnet-4-5-20250929                       │
│ ├── Stop Reason: tool_use                                   │
│ └── Content (8 blocks)                                      │
│     ├── Block 1                                             │
│     │   └── Tool Use: get_expenses                          │
│     │       ├── ID: toolu_01Nq2au3W69RmDFZaSdqe6u1          │
│     │       ├── Caller: code execution environment          │
│     │       └── Input:                                      │
│     │           └── {                                       │
│     │                 "employee_id": "ENG007",              │
│     │                 "quarter": "Q3"                       │
│     │               }                                       │
│     ├── Block 2                                             │
│     │   └── Tool Use: get_expenses                          │
│     │       ├── ID: toolu_01YYw9cuTSXk7bu7P38qBz6P          │
│     │       ├── Caller: code execution environment          │
│     │       └── Input:                                      │
│     │           └── {                                       │
│     │                 "employee_id": "ENG005",              │
│     │                 "quarter": "Q3"                       │
│     │               }                                       │
│     ├── Block 3                                             │
│     │   └── Tool Use: get_expenses                          │
│     │       ├── ID: toolu_01Fyxxe2KmVpVmw4jJL2CXSz          │
│     │       ├── Caller: code execution environment          │
│     │       └── Input:                                      │
│     │           └── {                                       │
│     │                 "employee_id": "ENG008",              │
│     │                 "quarter": "Q3"                       │
│     │               }                                       │
│     ├── Block 4                                             │
│     │   └── Tool Use: get_expenses                          │
│     │       ├── ID: toolu_01J4ovDu2UJa9Se19vxKTa6y          │
│     │       ├── Caller: code execution environment          │
│     │       └── Input:                                      │
│     │           └── {                                       │
│     │                 "employee_id": "ENG006",              │
│     │                 "quarter": "Q3"                       │
│     │               }                                       │
│     ├── Block 5                                             │
│     │   └── Tool Use: get_expenses                          │
│     │       ├── ID: toolu_01T24CrvQYA3LqGfZftCmueC          │
│     │       ├── Caller: code execution environment          │
│     │       └── Input:                                      │
│     │           └── {                                       │
│     │                 "employee_id": "ENG003",              │
│     │                 "quarter": "Q3"                       │
│     │               }                                       │
│     ├── Block 6                                             │
│     │   └── Tool Use: get_expenses                          │
│     │       ├── ID: toolu_01HotYZN6sk3gMLkpdWXbdz4          │
│     │       ├── Caller: code execution environment          │
│     │       └── Input:                                      │
│     │           └── {                                       │
│     │                 "employee_id": "ENG004",              │
│     │                 "quarter": "Q3"                       │
│     │               }                                       │
│     ├── Block 7                                             │
│     │   └── Tool Use: get_expenses                          │
│     │       ├── ID: toolu_01AxvEqi3AKqdnH44kGH1U6E          │
│     │       ├── Caller: code e


[Container] ID: container_011CVSAwq5J4vNPi3A4P2Rwh
[Container] Expires at: 2025-11-24 05:41:19.266670+00:00
[PTC] Tool called from code execution environment: get_expenses
[PTC] Tool called from code execution environment: get_expenses
[PTC] Tool called from code execution environment: get_expenses
[PTC] Tool called from code execution environment: get_expenses
[PTC] Tool called from code execution environment: get_expenses
[PTC] Tool called from code execution environment: get_expenses
[PTC] Tool called from code execution environment: get_expenses
[PTC] Tool called from code execution environment: get_expenses


╭─────────────────────────────────────── Claude API Response ───────────────────────────────────────╮
│ Claude Message (assistant) │ tokens: 4,751 in • 679 out • 5,430 total                             │
│ ├── Model: claude-sonnet-4-5-20250929                                                             │
│ ├── Stop Reason: tool_use                                                                         │
│ └── Content (6 blocks)                                                                            │
│     ├── Block 1                                                                                   │
│     │   └── Code Execution Result: Success (exit 0)                                               │
│     │       └── stdout:                                                                           │
│     │           └── Found 8 engineering team members                                              │
│     │                                                                                             │
│     │               Employees who exceeded $5,000 standard budget: 3                              │
│     │               ENG001: Alice Chen - $9177.88                                                 │
│     │               ENG003: Carol White - $6483.49                                                │
│     │               ENG007: Grace Taylor - $5289.35                                               │
│     │                                                                                             │
│     ├── Block 2                                                                                   │
│     │   └── Text                                                                                  │
│     │       └── Now let me check if any of these three employees have custom budget exceptions:   │
│     ├── Block 3                                                                                   │
│     │   └── Server Tool Use                                                                       │
│     │       ├── ID: srvtoolu_015GTpFmCbd2JPAQLAioB4Qb                                             │
│     │       ├── Caller: direct                                                                    │
│     │       └── Code:                                                                             │
│     │           └──    1                                                                          │
│     │                  2 import asyncio                                                           │
│     │                  3 import json                                                              │
│     │                  4                                                                          │
│     │                  5 async def main():                                                        │
│     │                  6     # Check custom budgets for the three employees who exceeded standard │
│     │                  7     exceeded_ids = ['ENG001', 'ENG003', 'ENG007']                        │
│     │                  8     exceeded_amounts = {                                                 │
│     │                  9         'ENG001': {'name': 'Alice Chen', 'travel_total': 9177.88},       │
│     │                 10         'ENG003': {'name': 'Carol White', 'travel_total': 6483.49},      │
│     │                 11         'ENG007': {'name': 'Grace Taylor', 'travel_total': 5289.35}      │
│     │                 12     }                                                                    │
│     │                 13                                                                          │
│     │                 14     # Get custom budgets in parallel                                     │
│     │                 15     budget_tasks = [                                                     │
│     │                 16         get_custom_budget({'user_id': emp_id})                           │
│     │               


[Container] ID: container_011CVSAwq5J4vNPi3A4P2Rwh
[Container] Expires at: 2025-11-24 05:41:33.430636+00:00
[PTC] Tool called from code execution environment: get_custom_budget
[PTC] Tool called from code execution environment: get_custom_budget
[PTC] Tool called from code execution environment: get_custom_budget


╭────────────────────────────────────────────── Claude API Response ──────────────────────────────────────────────╮
│ Claude Message (assistant) │ tokens: 5,611 in • 205 out • 5,816 total                                           │
│ ├── Model: claude-sonnet-4-5-20250929                                                                           │
│ ├── Stop Reason: end_turn                                                                                       │
│ └── Content (2 blocks)                                                                                          │
│     ├── Block 1                                                                                                 │
│     │   └── Code Execution Result: Success (exit 0)                                                             │
│     │       └── stdout:                                                                                         │
│     │           └── ENGINEERING TEAM MEMBERS WHO EXCEEDED THEIR Q3 TRAVEL BUDGET:                               │
│     │               ================================================================================            │
│     │                                                                                                           │
│     │               Alice Chen (ENG001)                                                                         │
│     │                 Budget Limit: $5,000.00 (Standard)                                                        │
│     │                 Travel Spending: $9,177.88                                                                │
│     │                 Exceeded By: $4,177.88                                                                    │
│     │                                                                                                           │
│     │               Carol White (ENG003)                                                                        │
│     │                 Budget Limit: $5,000.00 (Standard)                                                        │
│     │                 Travel Spending: $6,483.49                                                                │
│     │                 Exceeded By: $1,483.49                                                                    │
│     │                                                                                                           │
│     │               Grace Taylor (ENG007)                                                                       │
│     │                 Budget Limit: $5,000.00 (Standard)                                                        │
│     │                 Travel Spending: $5,289.35                                                                │
│     │                 Exceeded By: $289.35                                                                      │
│     │                                                                                                           │
│     └── Block 2                                                                                                 │
│         └── Text                                                                                                │
│             └── ## Summary                                                                                      │
│                                                                                                                 │
│                 **Three engineering team members exceeded their Q3 travel budget:**                             │
│                                                                                                                 │
│                 1. **Alice Chen (ENG001)**                                                                      │
│                    - Budget: $5,000 (Standard)                                                                  │
│                    - Spent: $9,177.88                 

In [ ]:
print(f"\n{'=' * 60}")
print(f"结果: {result_ptc}")
print(f"\n{'=' * 60}")
print("性能指标:")
print(
    f"  对Claude的总API调用: {len([m for m in conversation_ptc if m['role'] == 'assistant'])}"
)
print(f"  使用的总token: {total_tokens_ptc:,}")
print(f"  总耗时: {elapsed_time_ptc:.2f}s")

## 性能比较

让我们比较传统工具调用和PTC之间的性能：

**关于API调用计数的说明：**您可能会注意到，在此示例中，PTC需要更多API调用。这是因为PTC编写了更加结构化、顺序的代码，遵循最佳实践——例如，将费用获取步骤与预算检查步骤分开。传统工具调用有时可以在单次轮次中批量操作，但代价是通过模型的上下文发送所有原始数据。来自PTC的token效率增益远远超过了轮次增加的最小成本，特别是在处理大型、元数据丰富的数据集时。

In [ ]:
import pandas as pd

# Create comparison dataframe
comparison_data = {
    "Metric": [
        "API Calls",
        "Total Tokens",
        "Elapsed Time (s)",
        "Token Reduction",
        "Time Reduction",
    ],
    "Traditional": [
        api_count_without_ptc,
        f"{total_tokens:,}",
        f"{elapsed_time:.2f}",
        "-",
        "-",
    ],
    "PTC": [
        api_count_with_ptc,
        f"{total_tokens_ptc:,}",
        f"{elapsed_time_ptc:.2f}",
        f"{((total_tokens - total_tokens_ptc) / total_tokens * 100):.1f}%",
        f"{((elapsed_time - elapsed_time_ptc) / elapsed_time * 100):.1f}%",
    ],
}

df = pd.DataFrame(comparison_data)
print(df.to_string(index=False))

          Metric Traditional    PTC
       API Calls           4      4
    Total Tokens     110,473 15,919
Elapsed Time (s)       35.38  34.88
 Token Reduction           -  85.6%
  Time Reduction           -   1.4%


## 关键要点

在此示例中，PTC通过三个核心功能展示了显著的性能改进：

### 1. 通过大型数据解析保留上下文
这是我们在工作流程中展示的主要好处。Claude编写了代码来获取和处理代码执行环境中的数百条费用明细项目。通过以编程方式处理这些数据，Claude解析了JSON，按状态过滤，按类别汇总金额，并对照预算限额进行比较——所有这些都无需通过模型的上下文窗口发送原始费用数据和元数据。这导致了**token使用的显著减少**。

### 2. 顺序依赖优化  
API具有顺序依赖：`get_custom_budget(user_id)`，只能在分析费用以识别谁超出标准5000美元预算后调用。在传统工具调用中，这需要多个往返——获取团队成员，为每个人获取费用，识别超出预算的人，然后逐一检查他们的自定义预算。使用PTC，Claude编写了代码，在代码执行环境中协调整个工作流程，在循环中进行程序化工具调用，并在调用之间维护状态。这将许多顺序API往返转换为更少的调用和更智能的协调。

### 3. 代码执行中的计算逻辑
模型不需要在脑海中跟踪和汇总数十条具有复杂元数据的费用，而是将算术和聚合逻辑委托给Python代码。这减少了模型的认知负荷，确保了精确计算，并使不相关的元数据（如收据URL和商户位置）完全脱离模型的上下文。

---

## 何时使用PTC

PTC在以下情况下最有益：

- **处理大型、元数据丰富的数据集**时，需要过滤、解析或聚合（如我们的费用分析，包含收据URL、审批链、商户详细信息等）
- **存在顺序依赖关系**时，其中一个工具调用依赖于先前调用的结果（如仅对超出标准限额的员工检查自定义预算）
- **需要多个工具调用**时，在相似实体之间按顺序或循环进行（检查每个团队成员的费用和预算）
- **计算逻辑**可以减少需要通过模型上下文的内容
- **工具是安全的**，可以进行程序化/重复执行，无需人工监督

## 结论

我们的团队费用分析展示了PTC的优势：**在处理大型、元数据丰富的数据集时显著减少上下文消耗**，以及**优化具有顺序依赖的工作流程**。通过允许Claude编写代码来协调工具调用并以编程方式处理结果，我们在保持准确性和洞察质量的同时实现了大幅token节省。

PTC对于涉及批量数据处理、具有丰富元数据的重复工具调用、依赖关系，或原始工具输出否则会污染模型上下文的场景特别有价值。

## 后续步骤

尝试将此模式适应您自己的用例：
- 具有顺序查找的财务数据分析和报告
- 基于初始扫描结果的多实体健康检查  
- 具有元数据的大型文件处理（CSV、JSON、XML解析）
- 具有后续查询的数据库查询结果聚合
- 基于初始结果的批量API操作与条件逻辑